In [2]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from plotnine import *   
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev
from sklearn.linear_model import LogisticRegressionCV 

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
link = 'https://drive.google.com/file/d/1wVlHcBzHKAWJbYNVhPQJQW88aSkT6ic3/view?usp=sharing'
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('PS_20174392719_1491204439457_log.csv') 

In [11]:
df = pd.read_csv ('PS_20174392719_1491204439457_log.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
df = pd.read_csv ('PS_20174392719_1491204439457_log.csv')

In [7]:
df.drop('nameOrig',axis=1,inplace=True)   #removing column with indefined format
df.drop('nameDest',axis=1,inplace=True)   #removing column with indefined format
X=df.drop('isFraud',axis=1)
Y=df['isFraud']
enc = OrdinalEncoder()
X["type"] = enc.fit_transform(X[["type"]])  #transforming the "type column from string to integers"

### Divide dataset after importing into Train/Cross Validation/Test & Testing various hypothesis with different degrees

In [8]:
rmse_array = []
for i in range (1,3,1):
    features = PolynomialFeatures(degree=i, include_bias=False) #Generate feature matrix consisting of all polynomial combinations
    X_poly = features.fit_transform(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X_poly, Y, test_size = 0.3, random_state=5)
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size = 0.5, random_state=5)
    reg_model = LogisticRegression()
    reg_model.fit(X_train, Y_train)
    prediction = reg_model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(Y_valid, prediction))
    rmse_array.append(rmse)
print(rmse_array)

[0.03133330259081958, 0.03614677901848205]


In [11]:
features = PolynomialFeatures(degree=1, include_bias=False) #Generate feature matrix consisting of all polynomial combinations
X_poly = features.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_poly, Y, test_size = 0.3, random_state=5)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size = 0.5, random_state=5)
reg_model = LogisticRegression()
reg_model.fit(X_train, Y_train)
prediction = reg_model.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test, prediction)))

0.03103088294190997


### Applying K-Fold & Stratified Sampling

In [13]:
samp_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
accuracy = []
  
for train_idx, test_idx in samp_kfold.split(X_poly, Y):
    x_train_fold, x_test_fold = X_poly[train_idx], X_poly[test_idx]
    y_train_fold, y_test_fold = Y[train_idx], Y[test_idx]
    reg_model = LogisticRegression()
    reg_model.fit(x_train_fold, y_train_fold)
    accuracy.append(reg_model.score(x_test_fold, y_test_fold))
  
# Print the output.
print(accuracy, '\n')
print('Maximum Accuracy = ', max(accuracy)*100, '%', '\n')
print('Minimum Accuracy = ', min(accuracy)*100, '%', '\n')
print('Average Accuracy = ', mean(accuracy)*100, '%', '\n')
print('Standard Deviation = ', stdev(accuracy))

[0.9982876550854837, 0.998242862217137, 0.9982680090905948, 0.9983253753956703, 0.998249934775297] 

Maximum Accuracy =  99.83253753956703 % 

Minimum Accuracy =  99.82428622171369 % 

Average Accuracy =  99.82747673128365 % 

Standard Deviation =  3.319463944689407e-05


### Applying Regularization

In [15]:
parameter = [1,0.1,0.001,0.0001]
X=df.drop('isFraud',axis=1)
Y=df['isFraud']
enc = OrdinalEncoder()
X["type"] = enc.fit_transform(X[["type"]])  #transforming the "type column from string to integers"
features = PolynomialFeatures(degree=1, include_bias=False) #Generate feature matrix consisting of all polynomial combinations
X_poly = features.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_poly, Y, test_size = 0.3, random_state=5)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size = 0.5, random_state=5)
reg_model = LogisticRegression(penalty='l1', C=1 ,solver='saga', max_iter=2)
reg_model.fit(X_train, Y_train)
prediction = reg_model.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test, prediction)))

0.0366076330568006
